In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pandasql

# Part 1 - aadhaar_data.csv data wrangling using SQL syntax

In [2]:
# Read in aadhaar_data.csv to a pandas dataframe.  

# Sunsequently, rename the columns by replacing spaces with underscores and setting all characters to lowercase, so the
# column names more closely resemble columns names one might find in a table.

filename = "../data/aadhaar_data.csv"
aadhaar_data = pd.read_csv(filename)

aadhaar_data.rename(columns = lambda x: x.replace(' ', '_').lower(), inplace=True)

aadhaar_data

,registrar,enrolment_agency,state,district,sub_district,pin_code,gender,age,aadhaar_generated,enrolment_rejected,residents_providing_email,residents_providing_mobile_number
0,Allahabad Bank,Tera Software Ltd,Jharkhand,Ranchi,Namkum,834003,M,63,0,1,0,1
1,Allahabad Bank,Tera Software Ltd,Jharkhand,Ranchi,Ranchi,834004,F,36,0,1,0,1
2,Allahabad Bank,Vakrangee Softwares Limited,Gujarat,Surat,Nizar,394380,M,10,1,0,0,0
3,Allahabad Bank,Vakrangee Softwares Limited,Himachal Pradesh,Kangra,Baijnath,176081,M,44,1,0,1,1
4,Allahabad Bank,Vakrangee Softwares Limited,Madhya Pradesh,Chhindwara,Pandhurna,480334,M,35,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
119993,Central Bank of India,IAP COMPANY Pvt. Ltd,Maharashtra,Mumbai,Mumbai,400065,M,40,1,0,0,1
119994,Central Bank of India,IAP COMPANY Pvt. Ltd,Maharashtra,Mumbai,Mumbai,400066,M,33,1,0,1,1
119995,Central Bank of India,IAP COMPANY Pvt. Ltd,Maharashtra,Mumbai,Mumbai,400067,F,32,1,0,1,1
119996,Central Bank of India,IAP COMPANY Pvt. Ltd,Maharashtra,Mumbai,Mumbai,400067,M,21,1,0,0,1


## Simple SQLite query

In [3]:
def select_first_50(df):
    """Select out the first 50 values for "registrar" and "enrolment_agency"
    in the aadhaar_data table using SQLite syntax. 
    
    Note that "enrolment_agency" is spelled with one l. Also, the order of the select does matter. 
    We select registrar then enrolment agency.
    """
    query = """SELECT 
        registrar, enrolment_agency 
    FROM
        df
    LIMIT
        50
    """
    # Execute the SQL command against the pandas frame
    aadhaar_solution = pandasql.sqldf(query.lower(), locals())
    return aadhaar_solution   

select_first_50(aadhaar_data).head()

,registrar,enrolment_agency
0,Allahabad Bank,Tera Software Ltd
1,Allahabad Bank,Tera Software Ltd
2,Allahabad Bank,Vakrangee Softwares Limited
3,Allahabad Bank,Vakrangee Softwares Limited
4,Allahabad Bank,Vakrangee Softwares Limited


## Complex SQLite query

In [4]:
def aggregate_query(df):
    """We write a query that will select from the aadhaar_data table how many men and how many women 
    over the age of 50 have had aadhaar generated for them in each district.
    
    Note 1:
    Select gender first and then district.
    
    Note 2:
    aadhaar_generated is a column in the Aadhaar Data that denotes the number who have had aadhaar generated 
    in each row of the table.
    """
    q = """
    SELECT 
        gender, district, sum(aadhaar_generated)
    FROM
        df
    WHERE
        age > 50
    GROUP BY
        gender, district
    """

    # Execute the SQL command against the pandas frame
    aadhaar_solution = pandasql.sqldf(q.lower(), locals())
    return aadhaar_solution

aggregate_query(aadhaar_data).head(250)

,gender,district,sum(aadhaar_generated)
0,F,Ahmadnagar,45
1,F,Ahmed Nagar,0
2,F,Ahmedabad,1
3,F,Ajmer,27
4,F,Akola,5
...,...,...,...
245,F,Yadgir,3
246,F,Yamuna Nagar,130
247,F,Yavatmal,38
248,M,Ahmadnagar,63


# Part 2 - Using REST APIs

last.fm API Account

- Application name:	Spyridon Theodoros Dellas
- API key:	84c4f0c6253b8d9cb16294500dc5670b
- Shared secret:	6ee874cb8bcc55304916e8c4942fbcb8
- Registered to:	SpyrosDellas

## Last.fm API method `geo.getTopArtists` documentation

Get the most popular artists on Last.fm by country

#### Params
- country (Required) : A country name, as defined by the ISO 3166-1 country names standard
- limit (Optional) : The number of results to fetch per page. Defaults to 50.
- page (Optional) : The page number to fetch. Defaults to first page.
- api_key (Required) : A Last.fm API key.

#### Sample Response

```html
<topartists country="Spain">
  <artist rank="1">
    <name>Coldplay</name>
    <playcount>3199</playcount>
    <mbid>cc197bad-dc9c-440d-a5b5-d52ba2e14234</mbid>
    <url>http://www.last.fm/music/Coldplay</url>
    <streamable>1</streamable>
    <image size="small">...</image>
    <image size="medium">...</image>
    <image size="large">...</image>
  </artist>
  ...
</topartists>
```

In [5]:
import requests


def api_get_request(url):
    json = requests.get(url).json()
    top_artists = json['topartists']['artist']
    
    top_artist = None
    top_listeners = 0

    for artist in top_artists:
        name = artist['name']
        listeners = int(artist['listeners'])
        if listeners > top_listeners:
            top_listeners = listeners
            top_artist = name
    return top_artist


root_url = "http://ws.audioscrobbler.com/2.0/"
REST_query = "?method=geo.gettopartists&country=spain&api_key=84c4f0c6253b8d9cb16294500dc5670b&format=json"
url = root_url + REST_query
    
print(api_get_request(url))

Coldplay


## Data imputation

Pandas DataFrame has a method `fillna(value)`, such that you can pass in a single value to replace any NAs in a dataframe or series. 

We can call it like this:

```python
value = dataframe['column'].mean()
dataframe['column'].fillna(value, inplace=True)
```

`pd.Series.fillna`

```python
Signature:
pd.Series.fillna(
    self,
    value=None,
    method=None,
    axis=None,
    inplace=False,
    limit=None,
    downcast=None,
) -> Union[ForwardRef('Series'), NoneType]
Docstring:
Fill NA/NaN values using the specified method.

Parameters
----------
value : scalar, dict, Series, or DataFrame
    Value to use to fill holes (e.g. 0), alternately a
    dict/Series/DataFrame of values specifying which value to use for
    each index (for a Series) or column (for a DataFrame).  Values not
    in the dict/Series/DataFrame will not be filled. This value cannot
    be a list.
method : {'backfill', 'bfill', 'pad', 'ffill', None}, default None
    Method to use for filling holes in reindexed Series
    pad / ffill: propagate last valid observation forward to next valid
    backfill / bfill: use next valid observation to fill gap.
axis : {0 or 'index'}
    Axis along which to fill missing values.
inplace : bool, default False
    If True, fill in-place. Note: this will modify any
    other views on this object (e.g., a no-copy slice for a column in a
    DataFrame).
limit : int, default None
    If method is specified, this is the maximum number of consecutive
    NaN values to forward/backward fill. In other words, if there is
    a gap with more than this number of consecutive NaNs, it will only
    be partially filled. If method is not specified, this is the
    maximum number of entries along the entire axis where NaNs will be
    filled. Must be greater than 0 if not None.
downcast : dict, default is None
    A dict of item->dtype of what to downcast if possible,
    or the string 'infer' which will try to downcast to an appropriate
    equal type (e.g. float64 to int64 if possible).

Returns
-------
Series or None
    Object with missing values filled or None if ``inplace=True``.

See Also
--------
interpolate : Fill NaN values using interpolation.
reindex : Conform object to new index.
asfreq : Convert TimeSeries to specified frequency.

Examples
--------
>>> df = pd.DataFrame([[np.nan, 2, np.nan, 0],
...                    [3, 4, np.nan, 1],
...                    [np.nan, np.nan, np.nan, 5],
...                    [np.nan, 3, np.nan, 4]],
...                   columns=list('ABCD'))
>>> df
     A    B   C  D
0  NaN  2.0 NaN  0
1  3.0  4.0 NaN  1
2  NaN  NaN NaN  5
3  NaN  3.0 NaN  4

Replace all NaN elements with 0s.

>>> df.fillna(0)
    A   B   C   D
0   0.0 2.0 0.0 0
1   3.0 4.0 0.0 1
2   0.0 0.0 0.0 5
3   0.0 3.0 0.0 4

We can also propagate non-null values forward or backward.

>>> df.fillna(method='ffill')
    A   B   C   D
0   NaN 2.0 NaN 0
1   3.0 4.0 NaN 1
2   3.0 4.0 NaN 5
3   3.0 3.0 NaN 4

Replace all NaN elements in column 'A', 'B', 'C', and 'D', with 0, 1,
2, and 3 respectively.

>>> values = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
>>> df.fillna(value=values)
    A   B   C   D
0   0.0 2.0 2.0 0
1   3.0 4.0 2.0 1
2   0.0 1.0 2.0 5
3   0.0 3.0 2.0 4

Only replace the first NaN element.

>>> df.fillna(value=values, limit=1)
    A   B   C   D
0   0.0 2.0 2.0 0
1   3.0 4.0 NaN 1
2   NaN 1.0 NaN 5
3   NaN 3.0 NaN 4
```